In [2]:
import numpy as np
from scipy.stats import special_ortho_group
import itertools

np.random.seed(0)
d = 2
seed = 0

n way/mode Tensor:  
$\mathcal{T}_{{i_1}\cdots{i_n}} \to \mathcal{T'}_{{i_1}\cdots{i_n}} = (R_{{i_1}{j_1}})\cdots(R_{{i_n}{j_n}})\mathcal{T}_{j_1}\cdots{j_n}$  

 $v$ is a d dimension vector, to get n way tensor ($R^d \to (R^d)^n$)  
funtion multi_outer:  
 $\mathcal{T} = v \circ v \circ \cdots \circ v$  
It is obviously that $\mathcal{T}$ is a n way Tensor:

In [3]:
def multi_outer(v, n):
    out = v.copy()
    for _ in range(n - 1):
        out = out[..., None] * v
    return out

n = 4
v1 = np.random.randn(d)
T1 = multi_outer(v1, n)

R = special_ortho_group.rvs(dim=d)
v2 = R @ v1 
T2 = multi_outer(v2, n)

ranges = [[i for i in range(d)] for _ in range(n)]
T3 = np.zeros_like(T2)
for indices_i in itertools.product(*ranges):
    for indices_j in itertools.product(*ranges):
        T3[indices_i] += np.prod(R[indices_i, indices_j]) * T1[indices_j]

print(np.allclose(T2, T3, rtol=1e-8))


True


Similar to MTP, we can use a symmetric matrix to connect Tensors:  
  $f(\mathcal{T}_1, \mathcal{T}_1, \cdots) = \mathcal{T'}$  
For example, given:  

  $M = \begin{bmatrix} 0 &1 &1 \\1 &0 &2 \\ 1 &2 &0 \end{bmatrix} $ 

This correspond to three tensors (sum of the row): 2 way $\mathcal{T^1}$, 3 way $\mathcal{T^2}$, and 3 way $\mathcal{T^2}$

We can use part of the matrix such as the first two rows of M with 4 independent indices:

  $M_1 = \begin{bmatrix} 0 &x_1 &x_2 \\x_1 &0 &(x_3, x_4) \end{bmatrix} $ 

Then, we can connect a 2 way $\mathcal{T^1}$ and 3 way $\mathcal{T^2}$ and get a 3 way $\mathcal{T^{1\bigotimes2}}$ as:  
  $\mathcal{T^{1\bigotimes2}}_{{x_2}{x_3}{x_4}} = \mathcal{T^1}_{{x_1}{x_2}} \cdot \mathcal{T^2}_{{x_1}{x_3}{x_4}} $  
It is obviously that:
  $\mathcal{T^1}_{{x_1}{x_2}} \cdot \mathcal{T^2}_{{x_1}{x_3}{x_4}} \to (R_{{x_1}{y_1}})(R_{{x_2}{y_2}})\mathcal{T^1}_{{y_1}{y_2}}\cdot(R_{{x_1}{y_3}})(R_{{x_3}{y_4}})(R_{{x_4}{y_5}})\mathcal{T^2}_{{y_3}{y_4}{y_5}} $  
Noted that:  
  $(R_{{x_1}{y_1}})(R_{{x_1}{y_3}}) = \delta_{{y_1}{y_3}}$  
We have:  
$\mathcal{T^1}_{{x_1}{x_2}} \cdot \mathcal{T^2}_{{x_1}{x_3}{x_4}} \to (R_{{x_2}{y_2}})\mathcal{T^1}_{{y_1}{y_2}}\cdot(R_{{x_3}{y_4}})(R_{{x_4}{y_5}})\mathcal{T^2}_{{y_1}{y_4}{y_5}} $ 

And:  
 $\mathcal{T^{1\bigotimes2}}_{{x_2}{x_3}{x_4}} \to (R_{{x_2}{y_2}})(R_{{x_3}{y_3}})(R_{{x_4}{y_4}})\mathcal{T^{1\bigotimes2}}_{{y_2}{y_3}{y_4}}$

 The mode of new tensor is sum of the unused tensor (1 + 2 + 0 for above example)

If we use the last two row:  
  $M_1 = \begin{bmatrix} x_1 &0 &(x_3, x_4) \\x_2 &(x_3, x_4) & 0 \end{bmatrix} $  
   
We have:
  $\mathcal{T^{2\bigotimes3}}_{{x_1}{x_2}} = \mathcal{T^2}_{{x_1}{x_3}{x_4}} \cdot \mathcal{T^3}_{{x_2}{x_3}{x_4}} $  

It is easy to see it is a 2 way tensor (1 + 1 + 0).

To simplify the connect, if we have a x way tensor $\mathcal{T^1}$ and a y way tensor $\mathcal{T^2}$, we use the last two row of :   
$M = \begin{bmatrix} 0 & x - z & y - z \\x - z &0 &z \\ y - z &z &0 \end{bmatrix} $,  
 where $z \leq min(x, y)$ to connect them and the new
$\mathcal{T^{1\bigotimes2}}$ has way $x + y - 2z$  
Or:  

$\mathcal{T^{1\bigotimes2}}_{{a_1}\cdots{a_{x-z}}{b_1}\cdots{b_{y-z}}} = \mathcal{T^1}_{{a_1}\cdots{a_{x-z}}{c_1}\cdots{c_{z}}}\cdot\mathcal{T^2}_{{c_1}\cdots{c_{z}}{b_1}\cdots{b_{y-z}}}$


We firt check the new tensor got by matrix is same as the definition.  
We first expand $\mathcal{T^1}$ to way: $x + y - z$, we will have:  
$\mathcal{T}_{{a_1}\cdots{a_{x-z}}{c_1}\cdots{c_{z}}{b_1}\cdots{b_{y-z}}}$  
the new (or none when $y=z$) axis will repeat when multiplied by $\mathcal{T^2}$ with the broadcast in numpy.  
And we sum up the x-z way to z way

In [22]:

def tensor_connect(T1, T2, z):
    way1, way2 = len(T1.shape), len(T2.shape)
    T = T1.copy()
    while len(T.shape) < way1 + way2 - z:
        T = np.expand_dims(T, -1)
    T = np.sum(T * T2, axis=tuple([i for i in range(way1 - z, way1)]))
    return T


def tensor_connect_definition(T1, T2, z):
    shape1, shape2 = T1.shape, T2.shape
    way1, way2 = len(shape1), len(shape2)
    shape3 = (*shape1[:way1-z], *shape2[:way2-z])  # The shape of the new tensor

    T = np.zeros(shape3)
    ranges  = [[i for i in range(n)] for n in T.shape]  
    ranges1 = [[i for i in range(n)] for n in T1.shape[way1-z:]]  
    for indices in itertools.product(*ranges):     # a_1 ... a_{x-z} b_1 ... b_{y-z} 
        for ii in itertools.product(*ranges1):     # c_1 ... c_z
            i1 = (*indices[:way1-z], *ii)          # a_1 ... a_{x-z}  c_1 ... c_z
            i2 = (*ii, *indices[way1-z:])          # c_1 ... c_z  b_1 ... b_{y-z}  
            T[indices] += T1[i1] * T2[i2]
    return T

T1 = np.random.randn(3, 3, 3, 3)
T2 = np.random.randn(3, 3, 3)

for z in range(4):
    a = tensor_connect(T1, T2, z)
    b = tensor_connect_definition(T1, T2, z)
    if not np.allclose(a, b, rtol=1e-8):
        print(False)
else:
    print(True)
    
T1 = np.random.randn(3, 3, 3)
T2 = np.random.randn(3, 3, 3, 3)

for z in range(4):
    a = tensor_connect(T1, T2, z)
    b = tensor_connect_definition(T1, T2, z)
    if not np.allclose(a, b, rtol=1e-8):
        print(False)
else:
    print(True)

True
True


In [ ]:
from tensornet.utils import expand_to
import torch
import opt_einsum as oe
import time

def method1(input_tensor, filter_tensor, coupling_way):
    in_way = len(input_tensor.shape) - 4
    r_way = len(filter_tensor.shape) - 3
    n_way = in_way + r_way - coupling_way + 4
    input_tensor  = expand_to(input_tensor, n_way, dim=-1)
    filter_tensor = expand_to(filter_tensor, n_way, dim=3)

    sum_axis = [2] + [i for i in range(in_way - coupling_way + 4, in_way + 4)]
    output_tensor = torch.sum(input_tensor * filter_tensor, dim=sum_axis)
    return output_tensor


def method2(input_tensor, filter_tensor, coupling_way):
    in_way = len(input_tensor.shape) - 4
    r_way = len(filter_tensor.shape) - 3
    input_indices = [i for i in range(4, 4 + in_way - coupling_way)]
    coupling_indices = [i for i in range(4 + in_way - coupling_way, 4 + in_way)]
    filter_indices = [i for i in range(4 + in_way, 4 + in_way + r_way - coupling_way)]
    
    input_subscripts = [0, 1, 2, 3] + input_indices + coupling_indices
    filter_subscripts = [0, 1, 2] + coupling_indices + filter_indices
    output_subscripts = [0, 1, 3] + input_indices + filter_indices
    output_tensor = oe.contract(input_tensor, input_subscripts, filter_tensor, filter_subscripts, output_subscripts)
    return output_tensor

N = 10
nb, na, nn, nc, nd = 32, 64, 30, 64, 3
input_tensor = torch.randn(nb, na, nn, nc, )
filter_tensor = torch.randn(nb, na, nn, nd, nd)
coupling_way = 0
t = time.time()
for _ in range(N):
    x = method1(input_tensor, filter_tensor, coupling_way)
print(time.time() - t)
t = time.time()
for _ in range(N):
    y = method2(input_tensor, filter_tensor, coupling_way)
print(time.time() - t)

We can now define $F_n(r) \equiv f_n(r)\cdot r^{\bigotimes n}$ and use: 
${^n_i}T' = \sum\limits_{m, z} \sum\limits_{j\in \mathcal{N}(i)}{^m_j}\mathcal{T} \bigotimes^z F_{n-m+2z}(r) $
to update the n way tensors on the atoms *i*.  

We can check that the combinition of these tensors are equivalent to SO(n)

In [18]:
d = 4

def f(u, v):
    T1 = multi_outer(u, 4)
    T2 = multi_outer(v, 3)
    T3 = tensor_connect(T1, T2, 2)  # 3
    T4 = tensor_connect(T2, T3, 2)  # 2
    T5 = tensor_connect(T1, T4, 2)  # 2
    T6 = tensor_connect(T4, T5, 2)  # 0
    return T6

u = np.random.randn(d)
v = np.random.randn(d)
x = f(u, v)
R = special_ortho_group.rvs(dim=d)
u = R @ u
v = R @ v
y = f(u, v)

print(x - y)

1.3096723705530167e-10


In [19]:
d = 5

def f(u, v):
    T1 = multi_outer(u, 4)
    T2 = multi_outer(v, 3)
    T3 = tensor_connect(T1, T2, 2)  # 3
    T4 = tensor_connect(T2, T3, 2)  # 2
    T5 = tensor_connect(T1, T4, 2)  # 2
    T6 = tensor_connect(T5, u , 1)  # 1
    return T6

u = np.random.randn(d)
v = np.random.randn(d)
x = f(u, v)
R = special_ortho_group.rvs(dim=d)
u = R @ u
v = R @ v
y = f(u, v)

print(R @ x - y)

[ 1.38555833e-13  5.11590770e-13 -1.27897692e-13 -1.27897692e-13
 -5.96855898e-13]
